In [2]:
import re
import pandas as pd
import numpy as np
import math
import itertools

### Day 1

In [ ]:
f = open('../data/day_1_input.txt', 'r')

sum = 0

for line in f:
    nums = re.findall(r'\d', line)
    sum = sum + int(str(nums[0])+str(nums[-1]))

sum

In [ ]:
f = open('../data/day_1_input.txt', 'r')

sum = 0

digit_dict = {
    'one':1,
    'two':2,
    'three':3,
    'four':4,
    'five':5,
    'six':6,
    'seven':7,
    'eight':8,
    'nine':9
    }

for line in f:
    nums = re.findall(r'(?=(\d|one|two|three|four|five|six|seven|eight|nine))', line)
    for value in range(len(nums)):
        if nums[value] in digit_dict.keys():
            nums[value] = digit_dict[nums[value]]
    sum = sum + int(str(nums[0])+str(nums[-1]))

sum

### Day 2

In [ ]:
df = pd.DataFrame(columns = ['game_id', 'draw_id', 'red', 'blue', 'green'])

In [ ]:
f = open('../data/day_2_input.txt', 'r')

df = pd.DataFrame(columns = ['game_id', 'draw_id', 'red', 'blue', 'green'])

for line in f: 
    
    game_dict = {}

    colon = re.search(':', line).start()
    game_id = int(re.findall(r'\d+',line[:colon])[0])
    game_dict['game_id'] = game_id

    match = re.finditer(';', line)
    draws = [colon] + [m.start() for m in match] + [len(line)]

    for d in range(len(draws)-1):
        draw_str = line[draws[d]+1:draws[d+1]] 
        draw_id = d
        game_dict['draw_id'] = d

        colors = re.findall(r'blue|green|red', draw_str)
        scores = re.findall(r'\d+', draw_str)
        scores = [int(s) for s in scores]

        draw_dict = dict(zip(colors, scores))

        record_dict = game_dict.copy()
        record_dict.update(draw_dict)
        record_dict = {k:[record_dict[k]] for k in record_dict.keys()}
        
        df = pd.concat([df, pd.DataFrame.from_records(record_dict)])

final_df = (
    df
    .fillna(0)
    .assign(is_possible = lambda x: np.where((x['red']<=12) & (x['green']<=13) & (x['blue'] <= 14), 1, 0))
    .groupby('game_id')
    .agg({'is_possible':'min'})
    .reset_index()
    .loc[lambda x: x['is_possible']==1, :]
)

final_df['game_id'].sum()

In [ ]:
final_df = (
    df
    .fillna(0)
    .groupby('game_id')
    .max()
    .reset_index()
    .assign(power = lambda x: x['red']*x['blue']*x['green'])
)

final_df['power'].sum()

### Day 3

In [ ]:
f = open('../data/day_3_input.txt', 'r')

symbol_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'[^A-Za-z0-9.\n]', line)
    symbol_dict[index] = [m.start() for m in match_symbol]
    index = index + 1


In [ ]:
f = open('../data/day_3_input.txt', 'r')

gear_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'\*', line)
    gear_dict[index] = [m.start() for m in match_symbol]
    index = index + 1

gear_dict

In [ ]:
f = open('../data/day_3_input.txt', 'r')

index = 0
part_sum = 0

for line in f:
    match_part = re.finditer(r'\d+', line)
    parts = [(m.start(), m.end()) for m in match_part]
    for p in parts:
        is_valid = 0
        value = int(line[p[0]:p[1]])
        if p[0] - 1 in symbol_dict[index]:
            is_valid = 1
        if p[1] in symbol_dict[index]:
            is_valid = 1
        if index < 139:
            for x in symbol_dict[index + 1]:
                if x >= p[0] - 1 and x <= p[1]:
                    is_valid = 1
        if index > 1:
            for x in symbol_dict[index - 1]:
                if x >= p[0] - 1 and x <= p[1]:
                    is_valid = 1
        if is_valid == 1:
            part_sum = part_sum + value
    index = index + 1

part_sum

In [ ]:
f = open('../data/day_3_input.txt', 'r')

part_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'\d+', line)
    part_dict[index] = [(m.start(), m.end(), int(line[m.start():m.end()])) for m in match_symbol]
    index = index + 1

In [ ]:
f = open('../data/day_3_input.txt', 'r')

gear_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'\*', line)
    gear_dict[index] = [m.start() for m in match_symbol]
    index = index + 1

In [ ]:
gear_sum = 0

for line_index in range(139):
    for i in range(len(gear_dict[line_index])):
        matches = 0
        match_prod = 1
        # match_left
        end_index = [m[1] for m in part_dict[line_index]]
        if gear_dict[line_index][i] in end_index:
            part_value = part_dict[line_index][end_index.index(gear_dict[line_index][i])][2]
            matches = matches + 1
            match_prod = match_prod*part_value
        # match_right
        start_index = [m[0] for m in part_dict[line_index]]
        if gear_dict[line_index][i] + 1 in start_index:
            part_value = part_dict[line_index][start_index.index(gear_dict[line_index][i] + 1)][2]
            matches = matches + 1
            match_prod = match_prod*part_value
        # matches_above
        for m in part_dict[line_index - 1]:
            if not (m[0] >= gear_dict[line_index][i] + 2 or m[1] <= gear_dict[line_index][i] - 1):
                matches = matches + 1
                match_prod = match_prod*m[2]

        # matches_below
        for m in part_dict[line_index + 1]:
            if not (m[0] >= gear_dict[line_index][i] + 2 or m[1] <= gear_dict[line_index][i] - 1):
                matches = matches + 1
                match_prod = match_prod*m[2]

        if matches == 2:
            gear_sum = gear_sum + match_prod

gear_sum


### Day 4

In [ ]:
f = open('../data/day_4_input.txt', 'r')

total_score = 0

for line in f:

    winners = [int(x) for x in re.findall(r'\d+', line[re.search(":", line).end():re.search("\|", line).start()])]
    candidates = [int(x) for x in re.findall(r'\d+', line[re.search("\|", line).end():])]

    matches = 0
    for x in candidates:
        if x in winners:
            matches = matches + 1

    if matches == 0:
        points = 0
    else:
        points = 2**(matches - 1)

    total_score = total_score + points

total_score

In [ ]:
f = open('../data/day_4_input.txt', 'r')

total_score = 0
card_dict = {}

for line in f:

    card_id = int(re.findall(r'\d+', line[:re.search(":", line).start()])[0])
    winners = [int(x) for x in re.findall(r'\d+', line[re.search(":", line).end():re.search("\|", line).start()])]
    candidates = [int(x) for x in re.findall(r'\d+', line[re.search("\|", line).end():])]

    matches = 0
    for x in candidates:
        if x in winners:
            matches = matches + 1

    new_cards = [x+1 for x in range(card_id, card_id + matches)]
    card_dict[card_id] = new_cards

card_list = {x + 1:1 for x in range(206)}

for x in card_list.keys():
    for num in card_list.keys():
        if num in card_dict[x]:
            card_list[num] = card_list[num] + card_list[x]

sum(card_list.values())

### Day 5

In [ ]:
f = open('../data/day_5_input.txt', 'r').read()

def data_map(input, map_df):
    for i in range(map_df.shape[0]):
        if input in range(map_df[i][1], map_df[i][1]+map_df[i][2]):
            return int(map_df[i][0] + input - map_df[i][1])
    return int(input)

cats = [
    'seeds', 
    'seed-to-soil map',
    'soil-to-fertilizer map', 
    'fertilizer-to-water map', 
    'water-to-light map', 
    'light-to-temperature map', 
    'temperature-to-humidity map',
    'humidity-to-location map'
    ]

data_dict = {}

for i in range(8):
    if i == 0:
        data_dict[cats[i]] =  [int(x) for x in re.findall(r'\d+', f[re.search(cats[i], f).end():re.search(cats[i+1], f).start()])]
    else:
        if i < 7:
            data_dict[cats[i]] = np.array([int(x) for x in re.findall(r'\d+', f[re.search(cats[i], f).end():re.search(cats[i+1], f).start()])]).reshape((-1, 3))
        else:
            data_dict[cats[i]] = np.array([int(x) for x in re.findall(r'\d+', f[re.search(cats[i], f).end():])]).reshape((-1, 3))


In [ ]:
min_output = 999999999999999999999999

for i in range(len(data_dict['seeds'])):

    input = data_dict['seeds'][i]

    for x in [x for x in data_dict.keys() if x != 'seeds']:
        output = data_map(input, data_dict[x])
        input = output

    if output < min_output:
        min_output = output

print(min_output)


In [ ]:
seed_list = []
for i in range(int(len(data_dict['seeds'])/2)):
    seed_list.append(range(data_dict['seeds'][2*i], data_dict['seeds'][2*i] + data_dict['seeds'][2*i + 1]))

In [ ]:
# Returning a minimum with a huge step size over all possible seeds. 
# In this case, the square root of the length of each range
min_output = 999999999999999999999999
min_seed = 999999999999999999999999

for i in range(len(seed_list)):
    for j in range(0, len(seed_list[i]), math.floor(math.sqrt(len(seed_list[i])))):
        input = seed_list[i][j]

        for x in [x for x in data_dict.keys() if x != 'seeds']:
            output = data_map(input, data_dict[x])
            input = output

        if output < min_output:
            min_i = i
            min_j = j
            min_seed = seed_list[i][j]
            min_output = output

print('i: ', min_i)
print('j: ', min_j)
print('seed: ', min_seed)
print('output: ', min_output)


In [ ]:
# The step size used for the range that contains this approximate minimum
math.floor(math.sqrt(len(seed_list[5])))

In [ ]:
min_output = 999999999999999999999999
min_seed = 999999999999999999999999

# Fix the list that has the approximate minimum
i = 5

# Each the step size window around the approximate minimum
for j in range(121537743 - 11441, 121537743 + 11441):
    input = seed_list[i][j]

    for x in [x for x in data_dict.keys() if x != 'seeds']:
        output = data_map(input, data_dict[x])
        input = output

    if output < min_output:
        min_i = i
        min_j = j
        min_seed = seed_list[i][j]
        min_output = output

print('i: ', min_i)
print('j: ', min_j)
print('seed: ', min_seed)
print('output: ', min_output)

### Day 6

In [ ]:
f = open('../data/day_6_input.txt', 'r')

data_dict = {}

for line in f:
    print(line)
    data = line.lower().replace(':', '').split()
    data_dict[data[0]] = [int(x) for x in data[1:]]

data_dict

In [ ]:
counter_prod = 1

for race in range(len(data_dict['time'])):
    counter = 0
    for choice in range(data_dict['time'][race]+1):
        if choice*(data_dict['time'][race]-choice) > data_dict['distance'][race]:
            counter = counter + 1
            
    print(counter)
    counter_prod = counter_prod*counter

print(counter_prod)

In [ ]:
ms = ''
for i in range(len(data_dict['time'])):
    ms = ms+str(data_dict['time'][i])

ms = int(ms)

dist = ''
for i in range(len(data_dict['distance'])):
    dist = dist + str(data_dict['distance'][i])

dist = int(dist)

root_1 = (ms + math.sqrt(ms**2 - 4*dist))/2
root_2 = (ms - math.sqrt(ms**2 - 4*dist))/2

len(range(math.ceil(root_2), math.ceil(root_1)))


### Day 7

In [ ]:
hand_value_map = {
    (5, None):(15**3)*6,
    (4, 1):(15**3)*5,
    (3, 2):(15**3)*4,
    (3, 1):(15**3)*3,
    (2, 2):(15**3)*2,
    (2, 1):(15**3)*1,
    (1, 1):(15**3)*0
}

card_map = {
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10,
    'T':11,
    'J':12,
    'Q':13,
    'K':14,
    'A':15
}

card_map_jokers = {
    'J':1,
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10,
    'T':11,
    'Q':12,
    'K':13,
    'A':14
}

def hand_value(hand:str, joker = False):
    if joker == True:
        hand = [card_map_jokers[x] for x in list(hand)]
    else:
        hand = [card_map[x] for x in list(hand)]

    hand_grouped = {x:hand.count(x) for x in hand}

    if joker == True:
        wilds = hand_grouped.pop(1, 0)
        
    values_sorted = sorted(hand_grouped.values(), reverse=True)
    if len(values_sorted) > 0:
        largest_group = values_sorted[0]
    else:
        largest_group = 0
    
    if joker == True:
        largest_group = largest_group + wilds

    if largest_group == 5:
        second_largest_group = None
    else:
        second_largest_group = values_sorted[1]

    value = hand_value_map[(largest_group, second_largest_group)]

    tiebreaker = 0
    for i in range(5):
        tiebreaker = tiebreaker + (hand[i]-1)/15**(i-2)

    return value + tiebreaker

In [ ]:
f = open('../data/day_7_input.txt', 'r')

data_dict = {}

i = 0
for line in f:
    data = line.split()
    data_dict.update({i:{}})
    data_dict[i]['hand'] = data[0]
    data_dict[i]['bid'] = int(data[1])
    i = i + 1

df = pd.DataFrame.from_dict(data_dict, orient='index')

part_1 = (df
    .assign(value = df['hand'].apply(hand_value))
    .assign(rank = lambda x: x['value'].rank())
)

sum(part_1['rank']*part_1['bid'])

In [ ]:
part_2 = (df
    .assign(value = df['hand'].apply(hand_value, joker=True))
    .assign(rank = lambda x: x['value'].rank())
)

sum(part_2['rank']*part_2['bid'])

### Day 8

In [ ]:
f = open('../data/day_8_input.txt', 'r')

index = 0
nodes = {}
for line in f:
    if index == 0:
        instructions = line.strip()
    if index >= 2: 
        line = line.strip()
        idx = line.find('=')
        nodes[line[:idx-1]] = line[idx+2:].replace('(', '').replace(')', '').replace(' ', '').split(',')
    index = index + 1

In [ ]:
current_node = 'AAA'
step = 0
while current_node != 'ZZZ' and step < 100000: 
    direction = instructions[step % len(instructions)]
    if direction == 'L':
        current_node = nodes[current_node][0]
    else:
        current_node = nodes[current_node][1]
    step = step + 1

step

In [ ]:
for node in [x for x in nodes.keys() if x[2] == 'A']:
    current_node = node
    step = 0
    while step < 100000: 
        direction = instructions[step % len(instructions)]
        if direction == 'L':
            current_node = nodes[current_node][0]
        else:
            current_node = nodes[current_node][1]
        step = step + 1

        if current_node[2] == 'Z':
            print(step, step % len(instructions), node, current_node)

+ DNA - 20569
+ HNA - 18727
+ AAA - 14429
+ LMA - 13201 
+ VGA - 18113
+ LLA - 22411

In [ ]:
cycles = [20569, 18727, 14429, 13201, 18113, 22411]

In [ ]:
ans = math.lcm(20569, 18727)
for i in range(2,6):
    ans = math.lcm(ans, cycles[i])

ans

### Day 9

In [ ]:
f = open('../data/day_9_input.txt', 'r')

sequences = []
for line in f:
    sequences.append([int(x) for x in line.strip().split()])

In [ ]:
f = open('../data/day_9_input.txt', 'r')

sequences = []
for line in f:
    sequences.append([int(x) for x in line.strip().split()])

ans = 0

for pattern in sequences:
    last_entries = [pattern[len(pattern)-1]]
    diffs = np.array(pattern[1:]) - np.array(pattern[:-1]) 

    while ~np.all(diffs == 0):
        last_entries.append(diffs[len(diffs)-1])
        diffs = np.array(diffs[1:]) - np.array(diffs[:-1])


    ans = ans + sum(last_entries)

ans

In [ ]:
f = open('../data/day_9_input.txt', 'r')

sequences = []
for line in f:
    sequences.append([int(x) for x in line.strip().split()])

ans = 0

for pattern in sequences:
    first_entries = [pattern[0]]
    diffs = np.array(pattern[1:]) - np.array(pattern[:-1]) 
    
    idx = 1
    while ~np.all(diffs == 0):
        first_entries.append(((-1)**idx)*diffs[0])
        diffs = np.array(diffs[1:]) - np.array(diffs[:-1])
        idx = idx + 1

    ans = ans + sum(first_entries)

ans

### Day 10

In [ ]:
f = open('../data/day_10_input.txt', 'r')

grid = []

for line in f:
    grid.append(list(line.strip()))

steps_away = np.zeros(np.array(grid).shape)

In [ ]:
def connected_coords(x,y):
    if grid[y][x] == '|':
        return [[x, y+1], [x, y-1]]
    if grid[y][x] == '-':
        return [[x-1, y], [x+1, y]]
    if grid[y][x] == 'L':
        return [[x+1, y], [x, y-1]]
    if grid[y][x] == 'J':
        return [[x-1, y], [x, y-1]]
    if grid[y][x] == '7':
        return [[x-1, y], [x, y+1]]
    if grid[y][x] == 'F':
        return [[x+1, y], [x, y+1]] 
    else:
        return None

In [ ]:
current_coord = [88, 127]
prior_coord = [88, 128]
n_steps = 1

while grid[current_coord[1]][current_coord[0]] != 'S':
    steps_away[current_coord[1]][current_coord[0]] = n_steps
    for option in connected_coords(current_coord[0], current_coord[1]):
        if option != prior_coord:
            prior_coord = current_coord
            current_coord = option
            n_steps = n_steps + 1
            break


In [ ]:
current_coord = [88, 129]
prior_coord = [88, 128]
n_steps = 1

while grid[current_coord[1]][current_coord[0]] != 'S':
    if steps_away[current_coord[1]][current_coord[0]] != 0:
        if steps_away[current_coord[1]][current_coord[0]] > n_steps:
            steps_away[current_coord[1]][current_coord[0]] = n_steps
    for option in connected_coords(current_coord[0], current_coord[1]):
        if option != prior_coord:
            prior_coord = current_coord
            current_coord = option
            n_steps = n_steps + 1
            break


In [ ]:
steps_away.max()

### Day 11

In [ ]:
f = open('../data/day_11_input.txt', 'r')

grid = []

for line in f:
    grid.append(list(line.strip()))

grid = np.array(grid)

added_rows = 0
for row in range(grid.shape[0]):
    idx = row + added_rows
    if np.all(grid[idx] == '.'):
        grid = np.concatenate((grid[:idx], grid[idx].reshape((1, grid.shape[1])), grid[idx:]), axis = 0)
        added_rows = added_rows + 1

added_cols = 0
for col in range(grid.shape[1]):
    idx = col + added_cols
    if np.all(grid[:, idx] == '.'):
        grid = np.concatenate((grid[:, :idx], grid[:, idx].reshape((grid.shape[0],1)), grid[:, idx:]), axis = 1)
        added_cols = added_cols + 1

galaxies = []
for x in range(grid.shape[1]):
    for y in range(grid.shape[0]):
        if grid[y][x] == '#':
            galaxies.append((x, y))

ans = 0
for pair in itertools.combinations(galaxies, 2):
    ans = ans + abs(pair[0][0] - pair[1][0]) + abs(pair[0][1] - pair[1][1])

ans

In [ ]:
f = open('../data/day_11_input.txt', 'r')

grid = []

for line in f:
    grid.append(list(line.strip()))

grid = np.array(grid)

empty_rows = []
for row in range(grid.shape[0]):
    if np.all(grid[row] == '.'):
        empty_rows.append(row)

empty_cols = []
for col in range(grid.shape[1]):
    if np.all(grid[:, col] == '.'):
        empty_cols.append(col)


galaxies = []
for x in range(grid.shape[1]):
    for y in range(grid.shape[0]):
        if grid[y][x] == '#':
            galaxies.append((x, y))

ans = 0
expansion_factor = 999999
for pair in itertools.combinations(galaxies, 2):
    normal_rows = abs(pair[0][0] - pair[1][0])

    if pair[0][0] < pair[1][0]:
        extra_cols = expansion_factor*len([x for x in empty_cols if x in range(pair[0][0], pair[1][0])])
    else:
        extra_cols = expansion_factor*len([x for x in empty_cols if x in range(pair[1][0], pair[0][0])])

    normal_cols = abs(pair[0][1] - pair[1][1])

    if pair[0][1] < pair[1][1]:
        extra_rows = expansion_factor*len([x for x in empty_rows if x in range(pair[0][1], pair[1][1])])
    else:
        extra_rows = expansion_factor*len([x for x in empty_rows if x in range(pair[1][1], pair[0][1])])

    ans = ans + normal_rows + extra_rows + normal_cols + extra_cols

ans


### Day 12

In [ ]:
ques_options = ['.', '#']
chunk_mem = {}

f = open('../data/day_12_input.txt', 'r')

count = 0
for line in f:
    springs = line.strip().split(' ')[0]
    solution = [int(x) for x in line.strip().split(' ')[1].split(',')]

    min_runs = [x for x in springs.split('.') if x != '']
    chunk_options = []

    for run in range(len(min_runs)):
        chunk_options.append([])

        if min_runs[run] in chunk_mem.keys():
            chunk_options[run] = chunk_mem[min_runs[run]]
        else:
            choices = list(itertools.product([0, 1], repeat=min_runs[run].count('?')))
            replace_index = [i for i, ltr in enumerate(min_runs[run]) if ltr == '?']
            string_to_eval = min_runs[run]

        for choice in choices:
            for i in range(len(choice)):
                string_to_eval = string_to_eval[:replace_index[i]] + ques_options[choice[i]] + string_to_eval[replace_index[i]+1:]
            chunk_options[run].append([len(x) for x in string_to_eval.split('.') if x != ''])

    combs = [range(len(chunk_options[x])) for x in range(len(chunk_options))]
    combs = list(itertools.product(*combs))

    for x in combs:
        ans = []
        for i in range(len(chunk_options)):
            ans = ans + chunk_options[i][x[i]]
        if ans == solution:
            count = count + 1

count

In [ ]:
chunk_mem = {}
ques_options = ['.', '#']
count = 0

line = '?###???????? 3,2,1'
springs = 5*(line.strip().split(' ')[0]+'?')
springs = springs[:-1]
solution = 5*[int(x) for x in line.strip().split(' ')[1].split(',')]


min_runs = [x for x in springs.split('.') if x != '']
chunk_options = []

### Day 13

In [ ]:
f = open('../data/day_13_input.txt').read()

patterns = [x.split('\n') for x in f.split('\n\n')]

cols_above = 0
rows_left = 0

idx = 0
for pattern in patterns:
    for fold in range(1, len(pattern)):
        top_half = pattern[:fold]
        bottom_half = pattern[fold:]

        if len(top_half) > len(bottom_half):
            top_half = top_half[-len(bottom_half):]
        else:
            bottom_half = bottom_half[:len(top_half)]

        bottom_half.reverse()

        if top_half == bottom_half:
            cols_above = cols_above + fold

    for fold in range(1, len(pattern[0])):
        left_half = [x[:fold] for x in pattern]
        right_half = [x[fold:] for x in pattern]

        if len(left_half[0]) > len(right_half[0]):
            left_half = [x[-len(right_half[0]):] for x in left_half]
        else:
            right_half = [x[:len(left_half[0])] for x in right_half]

        right_half = [x[::-1] for x in right_half]

        if left_half == right_half:
            rows_left = rows_left + fold

    idx = idx + 1

100*cols_above + rows_left

In [ ]:
f = open('../data/day_13_input.txt').read()

patterns = [x.split('\n') for x in f.split('\n\n')]

# counting up the new mirrors
cols_above = 0
rows_left = 0

# correcting the smudges
for i in range(len(patterns)):

    new_pattern = np.array([list(x.replace('#', '1').replace('.', '0')) for x in patterns[i]]).astype(int)

    corrected = 0
    for fold in range(1, new_pattern.shape[0]): 
        top_half = new_pattern[:fold]
        bottom_half = new_pattern[fold:]

        if top_half.shape[0] > bottom_half.shape[0]:
            top_half = top_half[-bottom_half.shape[0]:]
        else:
            bottom_half = bottom_half[:top_half.shape[0]]

        bottom_half = bottom_half[::-1]

        diffs = ((top_half - bottom_half)**2)

        if diffs.sum() == 1:
            smudge = np.argwhere(diffs != 0)[0]
            spot_to_change = new_pattern[fold - diffs.shape[0] + smudge[0], smudge[1]]
            if spot_to_change == 0:
                new_pattern[fold - diffs.shape[0] + smudge[0], smudge[1]] = 1
            else:
                new_pattern[fold - diffs.shape[0] + smudge[0], smudge[1]] = 0
            corrected = 1
            cols_above = cols_above + fold
            break

    if corrected == 0:
        for fold in range(1, new_pattern.shape[1]):
            left_half = new_pattern[:, :fold]
            right_half = new_pattern[:, fold:]

            if left_half.shape[1] > right_half.shape[1]:
                left_half = left_half[:, -right_half.shape[1]:]
            else:
                right_half = right_half[:, :left_half.shape[1]]

            right_half = right_half[:, ::-1]

            diffs = ((left_half - right_half)**2)

            if diffs.sum() == 1:
                smudge = np.argwhere(diffs != 0)[0]
                spot_to_change = new_pattern[smudge[0], fold - diffs.shape[1] + smudge[1]]
                if spot_to_change == 0:
                    new_pattern[smudge[0], fold - diffs.shape[1] + smudge[1]] = 1
                else:
                    new_pattern[smudge[0], fold - diffs.shape[1] + smudge[1]] = 0
                rows_left = rows_left + fold
                break

100*cols_above + rows_left


### Day 14

In [ ]:
f = open('../data/day_14_input.txt').read()

platform = np.array([list(x) for x in f.replace('O', '1').replace('.', '0').replace('#', '9').split('\n')]).astype(int)

In [ ]:
def tilt(platform):

    new_platform = platform.copy()

    for col in range(platform.shape[1]):
        slice = platform[:, col]
        new_slice = platform[:, col].copy()

        rounds = np.where(platform[:, col] == 1)[0]
        cubes = np.where(platform[:, col] == 9)[0]
        cubes = [-1] + [x for x in cubes] + [platform.shape[0] + 1]

        for idx in range(len(cubes)-1):
            new_slice[cubes[idx]+1:cubes[idx+1]] = -np.sort(-new_slice[cubes[idx]+1:cubes[idx+1]])

        new_platform[:, col] = new_slice

    return new_platform

def cycle(platform):
    for i in range(4):
        platform = tilt(platform)
        platform = np.rot90(platform, 3)
    return platform

def north_load(platform):
    return (platform*np.array(range(platform.shape[0], 0, -1)).reshape((platform.shape[0], 1)))[platform==1].sum()


In [ ]:
north_load(tilt(platform))

In [ ]:
# Work in progress - to answer the problem I did this manually in Excel the first time. 

# Need to write a function that collapses my array to a string and back again
mem = {}
for i in range(1, 251):
    platform = cycle(platform)
    platform_str = platform.tobytes()
    if platform_str in mem:
        first_repeat = mem[platform_str]
        period = i - mem[platform_str]
        break
    else:
        mem[platform_str] = i

index = {}
for s, i in mem.items():
    index[i] = s

def cycle_value(cycle_count):
    return north_load(np.frombuffer(index[(first_repeat + (cycle_count - first_repeat) % period)], like=platform))

print(cycle_value(1000000000))

### Day 15

In [ ]:
def aoc_hash(string):
    current = 0
    for a in string:
        current = current + ord(a)
        current = 17*current
        current = current % 256
    return current

In [ ]:
f = open('../data/day_15_input.txt').read()

score = 0
for x in f.strip().split(','):
    score = score + aoc_hash(x)

In [ ]:
score

In [ ]:
f = open('../data/day_15_input.txt').read()

instructions = f.strip().split(',')

boxes = {x:{} for x in range(256)}

for sample in instructions:
    if sample[-1] == '-':
        action = '-'
        label = sample[:-1]
        focal_power = None
        box = aoc_hash(label)
        boxes[box].pop(label, None)
    else:
        action = '='
        label, focal_power = sample.split('=')
        box = aoc_hash(label)
        boxes[box].update({label:focal_power})

power = 0
for box in boxes:
    for i in range(len(boxes[box])):
        power = power + (box+1)*(i+1)*(int(list(boxes[box].values())[i]))

In [ ]:
power

### Day 16

In [ ]:
def step(beam_list):
    global visited
    global mem
    max_beam = max(beam_list.keys())
    new_beam_list = {}
    for idx in beam_list:

        beam = beam_list[idx]
        space = grid[beam[0][1]][beam[0][0]]

        if beam[0] not in visited:
            visited.append(beam[0])

        if beam not in mem:
            mem.append(beam)

        new_dir = ''
        new_loc = (-99, -99)

        if beam[1] == 'e':
            if space == '/':
                new_dir = 'n'
                new_loc = (beam[0][0], beam[0][1] - 1)
            elif space == '\\':
                new_dir = 's'
                new_loc = (beam[0][0], beam[0][1] + 1)
            elif space == '|':
                new_dir = 's'
                new_loc = (beam[0][0], beam[0][1] + 1)
                if beam[0][1] - 1 >= 0:
                    new_beam_list.update({max_beam+1:((beam[0][0], beam[0][1] - 1), 'n')})
                    max_beam = max_beam + 1 
            else:
                new_dir = 'e'
                new_loc = (beam[0][0] + 1, beam[0][1])    
        elif beam[1] == 'w':
            if space == '/':
                new_dir = 's'
                new_loc = (beam[0][0], beam[0][1] + 1)
            elif space == '\\':
                new_dir = 'n'
                new_loc = (beam[0][0], beam[0][1] - 1)
            elif space == '|':
                new_dir = 'n'
                new_loc = (beam[0][0], beam[0][1] - 1)
                if beam[0][1] + 1 <= len(grid) - 1:
                    new_beam_list.update({max_beam+1:((beam[0][0], beam[0][1] + 1), 's')})        
                    max_beam = max_beam + 1 
            else:
                new_dir = 'w'
                new_loc = (beam[0][0] - 1, beam[0][1]) 
        elif beam[1] == 'n':
            if space == '/':
                new_dir = 'e'
                new_loc = (beam[0][0] + 1, beam[0][1])
            elif space == '\\':
                new_dir = 'w'
                new_loc = (beam[0][0] - 1, beam[0][1])
            elif space == '-':
                new_dir = 'w'
                new_loc = (beam[0][0] - 1, beam[0][1])
                if beam[0][0] + 1 <= len(grid[0]) - 1:
                    new_beam_list.update({max_beam+1:((beam[0][0] + 1, beam[0][1]), 'e')}) 
                    max_beam = max_beam + 1 
            else:
                new_dir = 'n'
                new_loc = (beam[0][0], beam[0][1] - 1)
        elif beam[1] == 's':
            if space == '/':
                new_dir = 'w'
                new_loc = (beam[0][0] - 1, beam[0][1])
            elif space == '\\':
                new_dir = 'e'
                new_loc = (beam[0][0] + 1, beam[0][1])
            elif space == '-':
                new_dir = 'e'
                new_loc = (beam[0][0] + 1, beam[0][1])
                if beam[0][0] - 1 >= 0:
                    new_beam_list.update({max_beam+1:((beam[0][0] - 1, beam[0][1]), 'w')})
                    max_beam = max_beam + 1 
            else:
                new_dir = 's'
                new_loc = (beam[0][0], beam[0][1] + 1)
        if (new_loc, new_dir) not in mem and new_loc[0] >= 0 and new_loc[1] >= 0 and new_loc[1] <= len(grid) - 1 and new_loc[0] <= len(grid[0]) - 1:
            new_beam_list.update({idx:(new_loc, new_dir)}) 
    return new_beam_list

In [ ]:
f = open('../data/day_16_input.txt').read()

grid = f.split('\n')
beam_list = {1:((0,0), 'e')}
visited = []
mem = []
mem.append(beam_list[1])

n = 0
while len(beam_list) > 0 and n < 10000:
    beam_list = step(beam_list)
    n = n + 1
len(visited)

In [ ]:
ans = 0
for r in range(len(grid)):

  beam_list = {1:((0,r), 'e')}
  visited = []
  mem = []
  mem.append(beam_list[1])

  while len(beam_list):
      beam_list = step(beam_list)
  ans = max(ans, len(visited))

  beam_list = {1:((len(grid[0])-1,r), 'w')}
  visited = []
  mem = []
  mem.append(beam_list[1])

  while len(beam_list):
      beam_list = step(beam_list)
  ans = max(ans, len(visited))
  
  print('row', r, 'ans', ans)

for c in range(len(grid[0])):

  beam_list = {1:((c,0), 's')}
  visited = []
  mem = []
  mem.append(beam_list[1])

  while len(beam_list):
      beam_list = step(beam_list)
  ans = max(ans, len(visited))

  beam_list = {1:((c, len(grid)-1), 'n')}
  visited = []
  mem = []
  mem.append(beam_list[1])

  while len(beam_list):
      beam_list = step(beam_list)
  ans = max(ans, len(visited))

  print('col', c, 'ans', ans)


print(ans)

### Day 17

In [ ]:
f = open('../data/day_17_input.txt').read()

grid = f.split('\n')

score_grid = np.array([[int(x) for x in list(x)] for x in grid])

In [ ]:
current = (0,0, '')
nodes = (current, )
distances = {}

unvisited = {current: 0} #using None as +inf
visited = {}

currentDistance = unvisited[current]

letter_map = {
    (0,1):'r',
    (0,-1):'l',
    (1,0):'d',
    (-1,0):'u',
}

while True:
    avaialble_moves = [(0,1), (0, -1), (1, 0), (-1, 0)]
    # no more than three moves in a given direction
    if current[2] == 'rrr':
        avaialble_moves = [x for x in avaialble_moves if x != (0,1)]
    if current[2] == 'lll':
        avaialble_moves = [x for x in avaialble_moves if x != (0,-1)]
    if current[2] == 'ddd':
        avaialble_moves = [x for x in avaialble_moves if x != (1,0)]
    if current[2] == 'uuu':
        avaialble_moves = [x for x in avaialble_moves if x != (-1,0)]
    # can't directly reverse
    if current[2][-1:] == 'r':
        avaialble_moves = [x for x in avaialble_moves if x != (0,-1)]
    if current[2][-1:] == 'l':
        avaialble_moves = [x for x in avaialble_moves if x != (0,1)]
    if current[2][-1:] == 'd':
        avaialble_moves = [x for x in avaialble_moves if x != (-1,0)]
    if current[2][-1:] == 'u':
        avaialble_moves = [x for x in avaialble_moves if x != (1,0)]
    # boundaries
    if current[0] == len(grid)-1:
        avaialble_moves = [x for x in avaialble_moves if x != (1,0)]
    if current[0] == 0:
        avaialble_moves = [x for x in avaialble_moves if x != (-1,0)]
    if current[1] == len(grid[0])-1:
        avaialble_moves = [x for x in avaialble_moves if x != (0,1)]
    if current[1] == 0:
        avaialble_moves = [x for x in avaialble_moves if x != (0,-1)]

    neighbors = []
    for move in avaialble_moves:
        move_letter = letter_map[move]
        if move_letter == current[2][-1:]:
            recent_path = (current[2] + move_letter)[-3:]
        else:
            recent_path = move_letter
        neighbors = neighbors + [tuple(sum(x) for x in zip(current[:2],move)) + (recent_path, )]

    distances.update({current:{x:int(grid[x[0]][x[1]]) for x in neighbors}})

    for x in neighbors:
        if x not in nodes:
            nodes = nodes + (x,)
            unvisited.update({x:None})

    for neighbour, distance in distances[current].items():
        if neighbour not in unvisited: continue
        newDistance = currentDistance + distance
        if unvisited[neighbour] is None or unvisited[neighbour] > newDistance:
            unvisited[neighbour] = newDistance
    visited[current] = currentDistance
    if current[0] == len(grid)-1 and current[1] == len(grid[0])-1:
        print(current, currentDistance)
        break
    del unvisited[current]
    if not unvisited: break
    candidates = [node for node in unvisited.items() if node[1]]
    current, currentDistance = min(candidates, key = lambda x: x[1])

print(min([x[1] for x in visited.items() if x[0][0] == len(grid)-1 and x[0][1] == len(grid[0])-1]))


In [ ]:
current = (0,0, '')
nodes = (current, )
distances = {}

unvisited = {current: 0} #using None as +inf
visited = {}

currentDistance = unvisited[current]

letter_map = {
    (0,1):'r',
    (0,-1):'l',
    (1,0):'d',
    (-1,0):'u',
}

while True:
    avaialble_moves = []
    # no more than three moves in a given direction
    if current[2] == '':
        # right
        avaialble_moves = avaialble_moves + [(0, x) for x in range(4, 11)]
        # down
        avaialble_moves = avaialble_moves + [(x, 0) for x in range(4, 5)]
    if current[2] in ['r', 'l']:
        # down
        avaialble_moves = avaialble_moves + [(x, 0) for x in range(min(4, len(grid) - current[0]), min(11, len(grid) - current[0]))]
        # up
        avaialble_moves = avaialble_moves + [(-x, 0) for x in range(min(4, current[0]+1), min(11, current[0]+1))]
    if current[2] in ['u', 'd']:
        # right
        avaialble_moves = avaialble_moves + [(0, x) for x in range(min(4, len(grid[0]) - current[1]), min(11, len(grid[0]) - current[1]))]
        # left
        avaialble_moves = avaialble_moves + [(0, -x) for x in range(min(4, current[1]+1), min(11, current[1]+1))]

    neighbors = []
    for move in avaialble_moves:
        move_letter = letter_map[(max(min(move[0], 1), -1), max(min(move[1], 1), -1))]
        neighbors = neighbors + [tuple(sum(x) for x in zip(current[:2],move)) + (move_letter, )]

    distances.update({current:{x:score_grid[min(current[0], x[0]):max(current[0], x[0])+1, min(current[1], x[1]):max(current[1], x[1])+1].sum() - score_grid[current[0],current[1]] for x in neighbors}})

    for x in neighbors:
        if x not in nodes:
            nodes = nodes + (x,)
            unvisited.update({x:None})

    for neighbour, distance in distances[current].items():
        if neighbour not in unvisited: continue
        newDistance = currentDistance + distance
        if unvisited[neighbour] is None or unvisited[neighbour] > newDistance:
            unvisited[neighbour] = newDistance
    visited[current] = currentDistance
    if current[0] == len(grid)-1 and current[1] == len(grid[0])-1:
        print(current, currentDistance)
    del unvisited[current]
    if not unvisited: break
    candidates = [node for node in unvisited.items() if node[1]]
    current, currentDistance = min(candidates, key = lambda x: x[1])

print(min([x[1] for x in visited.items() if x[0][0] == len(grid)-1 and x[0][1] == len(grid[0])-1]))


Day 18

In [ ]:
import sys
sys.setrecursionlimit(100000)

def flood_fill(point):
    if grid[point[0], point[1]] == 1: return
    grid[point[0], point[1]] = 1
    flood_fill((point[0], point[1]+1))
    flood_fill((point[0], point[1]-1))
    flood_fill((point[0]+1, point[1]))
    flood_fill((point[0]-1, point[1]))
    return grid

In [ ]:
f = open('../data/day_18_input.txt').read()

instructions = [x.split(' ') for x in f.split('\n')]

grid = np.zeros((2000,2000), dtype=bool)

current = (1000, 1000)

for step in instructions:
    mult = 1
    direction = step[0]
    step_size = int(step[1])
    if direction in ('L', 'U'): mult = -1
    if direction in ('L', 'R'):
        new_loc = (current[0], current[1] + mult*step_size)
    else:
        new_loc = (current[0] + mult*step_size, current[1])

    grid[min(current[0], new_loc[0]):max(current[0], new_loc[0])+1, min(current[1], new_loc[1]):max(current[1], new_loc[1])+1] = True

    current = new_loc

grid = grid[:, np.sum(grid, axis=0) != 0]
grid = grid[np.sum(grid, axis=1) != 0, :]

inside_pt = (89, 1)

flood_fill(inside_pt).sum()

In [ ]:
f = open('../data/day_18_input.txt').read()

instructions = [x.split(' ') for x in f.split('\n')]

current = (0, 0)

verticies = [current]
boundary_points = 0

for step in instructions:
    mult = 1
    step_size = int(step[2].strip('()')[1:6], 16)
    direction = int(step[2].strip('()')[-1])
    if direction in (2, 3): mult = -1
    if direction in (0, 2):
        new_loc = (current[0], current[1] + mult*step_size)
    else:
        new_loc = (current[0] + mult*step_size, current[1])

    boundary_points = boundary_points + step_size
    verticies.append(new_loc)

    current = new_loc

x_coords = [x[0] for x in verticies]
y_coords = [x[1] for x in verticies]

area = (1/2)*abs(sum([x*y for x,y in zip(x_coords[:-1], y_coords[1:])]) - sum([x*y for x,y in zip(x_coords[1:], y_coords[:-1])]))
i = area - (boundary_points/2) + 1

print(boundary_points + i)


In [ ]:
# part one alt solution
f = open('../data/day_18_input.txt').read()

instructions = [x.split(' ') for x in f.split('\n')]

current = (0, 0)

verticies = [current]
boundary_points = 0

for step in instructions:
    mult = 1
    direction = step[0]
    step_size = int(step[1])
    if direction in ('L', 'U'): mult = -1
    if direction in ('L', 'R'):
        new_loc = (current[0], current[1] + mult*step_size)
    else:
        new_loc = (current[0] + mult*step_size, current[1])

    boundary_points = boundary_points + step_size
    verticies.append(new_loc)

    current = new_loc

x_coords = [x[0] for x in verticies]
y_coords = [x[1] for x in verticies]

area = (1/2)*abs(sum([x*y for x,y in zip(x_coords[:-1], y_coords[1:])]) - sum([x*y for x,y in zip(x_coords[1:], y_coords[:-1])]))
i = area - (boundary_points/2) + 1

print(boundary_points + i)



### Day 19

In [49]:
f = open('../data/day_19_input.txt').read()

In [50]:
things = f.split('\n\n')[1].split('\n')
things = [x.strip('\{\}').split(',') for x in things]
things = [{x[0]:int(x[2:]) for x in y} for y in things]

In [51]:
workflows = f.split('\n\n')[0].split('\n')
workflows = {x[:x.find('{')]:x[x.find('{'):].strip('}{').split(',') for x in workflows}
workflows = {k:[(x[0], x[1], int(x[2:x.find(':')]), x[x.find(':')+1:]) for x in v[:-1]] + [(v[-1], )] for k,v in workflows.items()}

In [52]:
def process_workflow(item, start_workflow):
    for step in workflows[start_workflow]:
        if len(step) == 1:
            return step[0]
        else:
            if step[1] == '<':
                if item[step[0]] < step[2]:
                    return step[3]
            if step[1] == '>':
                if item[step[0]] > step[2]:
                    return step[3]


In [53]:
sum_accepted = 0
for item in things:
    workflow = 'in'
    while workflow not in ('A', 'R'):
        workflow = process_workflow(item, workflow)
    if workflow == 'A':
        sum_accepted = sum_accepted + sum(item.values())

print(sum_accepted)

325952


In [54]:
graph_nodes = tuple(list(workflows.keys())+['A', 'R'])
graph_edges = []

for x, v in workflows.items():
    converse = ''
    for y in v[:-1]:
        graph_edges.append((x, y[3], y[0]+y[1]+str(y[2])))     
    graph_edges.append((x, v[-1][0], '='))

graph = {x:[y[1:] for y in graph_edges if y[0] == x] for x in graph_nodes}

In [70]:
tests = [('in',dict(zip('xmas',[range(1,4000)]*4)))]
ans = 0
while tests:
    origin,ranges = tests.pop(0)
    if origin == 'A':
        ans = ans + math.prod(len(x)+1 for x in ranges.values())
    elif origin != 'R':
        for dest,rule in graph[origin]:
            # create a tuple by splitting on the operator
            letter, operator, number, dest = (*re.split('([<=>])',rule), dest)
            if operator == '=':
                tests.append((dest,ranges.copy()))
                print(dest, ranges)
            elif operator == '>':
                letter_range = ranges.get(letter)
                # send relevant portion of the range to the next rule
                ranges[letter] = range(1+max(letter_range.start,int(number)),letter_range.stop)
                tests.append((dest,ranges.copy()))
                print(dest,ranges)
                # keep the remaining portion of the range here
                ranges[letter] = range(letter_range.start,min(letter_range.stop,int(number)))
            elif operator == '<':
                letter_range = ranges.get(letter)
                # send relevant portion of the range to the next rule
                ranges[letter] = range(letter_range.start,min(letter_range.stop,int(number))-1)
                tests.append((dest,ranges.copy()))
                print(dest,ranges)
                # keep the remaining portion of the range here
                ranges[letter] = range(max(letter_range.start,int(number)),letter_range.stop)
        
print(ans)



npv {'x': range(1, 4000), 'm': range(2049, 4000), 'a': range(1, 4000), 's': range(1, 4000)}
qmn {'x': range(1, 4000), 'm': range(1, 2048), 'a': range(1, 4000), 's': range(1, 4000)}
sd {'x': range(1, 4000), 'm': range(2887, 4000), 'a': range(1, 4000), 's': range(1, 4000)}
ztd {'x': range(1, 4000), 'm': range(2049, 2886), 'a': range(1, 2541), 's': range(1, 4000)}
dzh {'x': range(1, 4000), 'm': range(2412, 2886), 'a': range(2542, 4000), 's': range(1, 4000)}
dvr {'x': range(1, 4000), 'm': range(2049, 2411), 'a': range(2542, 4000), 's': range(1, 4000)}
xnh {'x': range(1, 2007), 'm': range(1, 2048), 'a': range(1, 4000), 's': range(1, 4000)}
jv {'x': range(2008, 4000), 'm': range(1, 2048), 'a': range(1, 2167), 's': range(1, 4000)}
msr {'x': range(2008, 4000), 'm': range(1, 2048), 'a': range(2168, 4000), 's': range(1, 4000)}
vz {'x': range(1, 4000), 'm': range(2887, 4000), 'a': range(1, 4000), 's': range(2600, 4000)}
qrq {'x': range(1, 4000), 'm': range(2887, 4000), 'a': range(1, 4000), 's': r

### Day 20

In [241]:
f = open('../data/day_20_input.txt').read().split('\n')
f = [x.split(' -> ') for x in f]

modules = {}
for x in f:
    if x[0] == 'broadcaster':
        modules.update({x[0]:x[1].split(', ')})
    elif x[0][0] == '%':
        modules.update({x[0][1:]:[x[0][0], x[1].split(', '), 0]})
    elif x[0][0] == '&':
        connections = {}
        for y in f: 
            if x[0][1:] in y[1].split(', '):
                connections.update({y[0][1:]:0})
        modules.update({x[0][1:]:[x[0][0], x[1].split(', '), connections]})

cycle_count = 0
high_count = 0
low_count = 0
press_button = True
queue = []

while press_button and cycle_count < 1000:

    queue.append(('button','broadcaster', 0))
    cycle_count = cycle_count + 1

    while queue:
        pulse = queue.pop(0)
        if pulse[1] == 'rx' and pulse[2] == 0:
            break
        if pulse[2] == 1: high_count = high_count + 1
        if pulse[2] == 0: low_count = low_count + 1
        if pulse[1] not in modules.keys():
            continue
        if pulse[1] == 'broadcaster':
            for dest in modules[pulse[1]]:
                queue.append((pulse[1], dest, 0))
        if modules[pulse[1]][0] == '%':
            if pulse[2] == 0:
                if modules[pulse[1]][2] == 0:
                    modules[pulse[1]][2] = 1
                    for dest in modules[pulse[1]][1]:
                        queue.append((pulse[1], dest, 1))
                elif modules[pulse[1]][2] == 1:
                    modules[pulse[1]][2] = 0
                    for dest in modules[pulse[1]][1]:
                        queue.append((pulse[1], dest, 0))
        if modules[pulse[1]][0] == '&':
            modules[pulse[1]][2][pulse[0]] = pulse[2]
            if all(value == 1 for value in modules[pulse[1]][2].values()):
                for dest in modules[pulse[1]][1]:
                    queue.append((pulse[1], dest, 0))
            else:
                for dest in modules[pulse[1]][1]:
                    queue.append((pulse[1], dest, 1))

    # press_button = False
    press_button = any([any(v[2].values()) for v in modules.values() if v[0] == '&']) or any([v[2] for v in modules.values() if v[0] == '%'])

print(high_count*low_count*(1000/cycle_count)**2)

787056720.0


In [14]:
f = open('../data/day_20_input.txt').read().split('\n')
f = [x.split(' -> ') for x in f]

modules = {}
for x in f:
    if x[0] == 'broadcaster':
        modules.update({x[0]:x[1].split(', ')})
    elif x[0][0] == '%':
        modules.update({x[0][1:]:[x[0][0], x[1].split(', '), 0]})
    elif x[0][0] == '&':
        connections = {}
        for y in f: 
            if x[0][1:] in y[1].split(', '):
                connections.update({y[0][1:]:0})
        modules.update({x[0][1:]:[x[0][0], x[1].split(', '), connections]})

# modules['broadcaster'] = ['js']

cycle_count = 0
high_count = 0
low_count = 0
press_button = True
queue = []

while press_button and cycle_count < 30000:

    queue.append(('button','broadcaster', 0))
    cycle_count = cycle_count + 1

    while queue:
        pulse = queue.pop(0)
        if pulse[1] == 'ql' and pulse[2] == 1:
            print(cycle_count, pulse)
        if pulse[2] == 1: high_count = high_count + 1
        if pulse[2] == 0: low_count = low_count + 1
        if pulse[1] not in modules.keys():
            continue
        if pulse[1] == 'broadcaster':
            for dest in modules[pulse[1]]:
                queue.append((pulse[1], dest, 0))
        if modules[pulse[1]][0] == '%':
            if pulse[2] == 0:
                if modules[pulse[1]][2] == 0:
                    modules[pulse[1]][2] = 1
                    for dest in modules[pulse[1]][1]:
                        queue.append((pulse[1], dest, 1))
                elif modules[pulse[1]][2] == 1:
                    modules[pulse[1]][2] = 0
                    for dest in modules[pulse[1]][1]:
                        queue.append((pulse[1], dest, 0))
        if modules[pulse[1]][0] == '&':
            modules[pulse[1]][2][pulse[0]] = pulse[2]
            if all(value == 1 for value in modules[pulse[1]][2].values()):
                for dest in modules[pulse[1]][1]:
                    queue.append((pulse[1], dest, 0))
            else:
                for dest in modules[pulse[1]][1]:
                    queue.append((pulse[1], dest, 1))

    # press_button = False
    press_button = any([any(v[2].values()) for v in modules.values() if v[0] == '&']) or any([v[2] for v in modules.values() if v[0] == '%'])

3761 ('mf', 'ql', 1)
3793 ('fz', 'ql', 1)
3847 ('fh', 'ql', 1)
3881 ('ss', 'ql', 1)
7522 ('mf', 'ql', 1)
7586 ('fz', 'ql', 1)
7694 ('fh', 'ql', 1)
7762 ('ss', 'ql', 1)
11283 ('mf', 'ql', 1)
11379 ('fz', 'ql', 1)
11541 ('fh', 'ql', 1)
11643 ('ss', 'ql', 1)
15044 ('mf', 'ql', 1)
15172 ('fz', 'ql', 1)
15388 ('fh', 'ql', 1)
15524 ('ss', 'ql', 1)
18805 ('mf', 'ql', 1)
18965 ('fz', 'ql', 1)
19235 ('fh', 'ql', 1)
19405 ('ss', 'ql', 1)
22566 ('mf', 'ql', 1)
22758 ('fz', 'ql', 1)
23082 ('fh', 'ql', 1)
23286 ('ss', 'ql', 1)
26327 ('mf', 'ql', 1)
26551 ('fz', 'ql', 1)
26929 ('fh', 'ql', 1)
27167 ('ss', 'ql', 1)


In [10]:
cycles = [3761, 3793, 3847, 3881]
ans = 1
for i in cycles:
    ans = math.lcm(ans, i)

ans

212986464842911

### Day 21

In [203]:
f = open('../data/day_21_input.txt').read()

grid = f.split('\n')

In [204]:
graph = {}

for row in range(len(grid)):
    for col in range(len(grid[row])): 
        if grid[row][col] == 'S': start = set([(row, col)])
        if grid[row][col] in ('.', 'S'):
            edges = []
            if row+1 < len(grid):
                if grid[row+1][col] in ('.', 'S'): 
                    edges.append((row+1, col))
            if row-1 >= 0:
                if grid[row-1][col] in ('.', 'S'): 
                    edges.append((row-1, col))
            if col+1 < len(grid[row]):
                if grid[row][col+1] in ('.', 'S'): 
                    edges.append((row, col+1))
            if col-1 >= 0:
                if grid[row][col-1] in ('.', 'S'): 
                    edges.append((row, col-1))
            graph.update({(row, col):edges})

def step(start, graph):
    end = []
    for plot in start:
        end = end + graph[plot]
    return set(end)

plots = start
step_count = 0

while step_count < 64:
    plots = step(plots, graph)
    step_count = step_count +1

print(len(plots))

3743


In [216]:
grid = {i+j*1j:c for i,r in enumerate(open('../data/day_21_input.txt')) for j,c in enumerate(r) if c in '.S'}

plots = {x for x in grid if grid[x]=='S'}
cmod = lambda x: complex(x.real%131, x.imag%131)

for s in range(3 * 131):
    if s == 64: print(s, len(plots))
    if s%131 == 65: print(s, len(plots))

    plots = {p+d for d in {1, -1, 1j, -1j} for p in plots if cmod(p+d) in grid}

64 3743
65 3847
196 34165
327 94697


In [201]:
import numpy as np

points = [(65, 3847), (196, 34165), (327, 94697)]

def evaluate_quadratic_equation(points, x):
    # Fit a quadratic polynomial (degree=2) through the points
    coefficients = np.polyfit(*zip(*points), 2)
    print(coefficients)

    # Evaluate the quadratic equation at the given x value
    result = np.polyval(coefficients, x)
    return round(result)

evaluate_quadratic_equation(points, 26501365)

[ 0.88031001  1.67420314 18.86702407]


618261433219147

### Day 22

In [53]:
f = open('../data/day_22_input.txt').read()

In [63]:
bricks = []
for l in f.split("\n"):
    sloc,eloc = l.split("~")
    sx,sy,sz = map(int,sloc.split(","))
    ex,ey,ez = map(int,eloc.split(","))
    bricks.append((sx,sy,sz,ex,ey,ez))

def is_solid(bset, x,y,z):
    if z == 0:
        return True
    return (x,y,z) in bset

def bfall(bricks):
    fell = False
    bset = set()
    for (sx,sy,sz,ex,ey,ez) in bricks:
        for x in range(sx,ex+1):
            for y in range(sy,ey+1):
                bset.add((x,y,ez))
    
    new_bricks = []
    for (sx,sy,sz,ex,ey,ez) in bricks:
        supp = False
        for x in range(sx,ex+1):
            for y in range(sy,ey+1):
                if is_solid(bset, x, y, sz - 1):
                    supp = True
                    break
            if supp:
                break
        if not supp:
            fell = True
            new_bricks.append((sx,sy,sz-1, ex,ey,ez-1))
        else:
            new_bricks.append((sx,sy,sz,ex,ey,ez))
    return fell, new_bricks


fell = True
cnt = 0
while fell:
    cnt += 1
    fell, bricks = bfall(bricks)

bcopy = bricks.copy()

ans = 0
for i in range(len(bcopy)):
    bcopy2 = bcopy.copy()
    del bcopy2[i]
    if not bfall(bcopy2)[0]:
        ans += 1
print(ans)

488


In [64]:
ans = 0
for i in range(len(bcopy)):
    bcopy2 = bcopy.copy()
    del bcopy2[i]
    bcopy3 = bcopy2.copy()
    fell = True
    while fell:
        fell, bcopy2 = bfall(bcopy2)
    mm = 0
    for a,b in zip(bcopy2, bcopy3):
        if a != b:
            mm += 1
    ans += mm
print(ans)

79465


In [59]:
bfall(bricks)

{(7, 7, 320),
 (5, 3, 177),
 (8, 4, 134),
 (5, 8, 292),
 (7, 0, 281),
 (3, 3, 265),
 (3, 7, 6),
 (6, 6, 187),
 (2, 8, 268),
 (3, 6, 47),
 (9, 7, 169),
 (5, 7, 183),
 (3, 7, 271),
 (5, 8, 148),
 (4, 3, 182),
 (4, 4, 147),
 (8, 2, 66),
 (6, 1, 108),
 (3, 0, 3),
 (0, 9, 28),
 (5, 2, 104),
 (6, 6, 43),
 (7, 8, 177),
 (8, 4, 255),
 (7, 1, 138),
 (9, 6, 246),
 (7, 7, 32),
 (2, 5, 235),
 (0, 6, 22),
 (1, 7, 134),
 (3, 4, 238),
 (9, 4, 56),
 (1, 0, 95),
 (4, 0, 149),
 (4, 6, 192),
 (5, 8, 269),
 (3, 3, 13),
 (2, 4, 126),
 (2, 3, 310),
 (6, 8, 237),
 (5, 9, 121),
 (2, 8, 16),
 (9, 2, 68),
 (7, 0, 294),
 (7, 1, 259),
 (7, 7, 153),
 (2, 6, 55),
 (0, 6, 143),
 (6, 8, 273),
 (1, 0, 216),
 (6, 1, 85),
 (2, 3, 166),
 (2, 8, 281),
 (0, 8, 189),
 (8, 2, 308),
 (6, 2, 315),
 (7, 6, 230),
 (7, 5, 265),
 (5, 1, 122),
 (3, 7, 104),
 (1, 7, 111),
 (2, 3, 22),
 (1, 6, 3),
 (5, 9, 242),
 (2, 8, 137),
 (8, 6, 198),
 (8, 5, 233),
 (7, 2, 196),
 (4, 7, 165),
 (1, 4, 222),
 (7, 8, 10),
 (4, 2, 315),
 (4, 3, 280),

In [14]:
tuple(map(int, [tuple(x.split('~')) for x in f.split('\n')][0][0].split(',')))

(1, 0, 1)